# TPE analysis and Tissue detection


In [ ]:
from pathlib import Path # For working with file paths
import mps  # For reading MPS data
import matplotlib.pyplot as plt # For plotting
import ap_features as apf

In [ ]:
tpe_data_path = Path("tpe.npy")

def download_data():
    print("Downloading data. Please wait...")
    link = "https://www.dropbox.com/s/qga5fi6xzt306f1/tpe.npy?dl=1"
    import urllib.request
    import time

    urllib.request.urlretrieve(link, motion_data_path)
    time.sleep(1.0)
    print("Done downloading data")
    
if not tpe_data_path.is_file():
    download_data()
    
data = mps.MPS(tpe_data_path)

In [ ]:
data.info

In [ ]:
first_frame = data.frames[:, :, 0].T

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))
ax.imshow(first_frame)

In [ ]:
x0 = (50, 600)
y0 = (50, 200)

x1 = (50, 600)
y1 = (220, 350)

x2 = (50, 600)
y2 = (400, 520)

x3 = (50, 600)
y3 = (550, 700)

fig, ax = plt.subplots()
ax.imshow(first_frame)
for (xi, yi) in [(x0, y0), (x1, y1), (x2, y2), (x3, y3)]:
    ax.plot([xi[0], xi[0]], yi, color="r")
    ax.plot([xi[1], xi[1]], yi, color="r")
    ax.plot(xi, [yi[0], yi[0]], color="r")
    ax.plot(xi, [yi[1], yi[1]], color="r")

In [ ]:
tissue0 = data.frames[x0[0]:x0[1], y0[0]:y0[1], :]
tissue1 = data.frames[x1[0]:x1[1], y1[0]:y1[1], :]
tissue2 = data.frames[x2[0]:x2[1], y2[0]:y2[1], :]
tissue3 = data.frames[x3[0]:x3[1], y3[0]:y3[1], :]

fig, ax = plt.subplots()
ax.imshow(tissue1[:, :, 0].T)
plt.show()

In [ ]:
y_tissue0 = tissue0.mean(axis=(0, 1))
y_tissue1 = tissue1.mean(axis=(0, 1))
y_tissue2 = tissue2.mean(axis=(0, 1))
y_tissue3 = tissue3.mean(axis=(0, 1))

fig, ax = plt.subplots()
ax.plot(data.time_stamps, y_tissue0, label="tissue0")
ax.plot(data.time_stamps, y_tissue1, label="tissue1")
ax.plot(data.time_stamps, y_tissue2, label="tissue2")
ax.plot(data.time_stamps, y_tissue3, label="tissue3")
ax.legend()
plt.show()

In [ ]:
trace_tissue0 = apf.Beats(y=y_tissue0, t=data.time_stamps, background_correction_method="full")
trace_tissue1 = apf.Beats(y=y_tissue1, t=data.time_stamps, background_correction_method="full")
trace_tissue2 = apf.Beats(y=y_tissue2, t=data.time_stamps, background_correction_method="full")
trace_tissue3 = apf.Beats(y=y_tissue3, t=data.time_stamps, background_correction_method="full")

fig, ax = plt.subplots()
ax.plot(trace_tissue0.t, trace_tissue0.y, label="tissue0")
ax.plot(trace_tissue1.t, trace_tissue1.y, label="tissue1")
ax.plot(trace_tissue2.t, trace_tissue2.y, label="tissue2")
ax.plot(trace_tissue3.t, trace_tissue3.y, label="tissue3")
ax.legend()
plt.show()

In [ ]:
mask = mps.tissue_detection.detect_tissue(frame=first_frame, sigma=20.0, min_size=400, fill_holes=False)

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
ax[0].imshow(first_frame)
ax[0].imshow(mask, alpha=0.1, cmap="gray")
ax[1].imshow(mask, cmap="gray")

In [ ]:
mask_tissue0 = mask.T[x0[0]:x0[1], y0[0]:y0[1]]
mask_tissue1 = mask.T[x1[0]:x1[1], y1[0]:y1[1]]
mask_tissue2 = mask.T[x2[0]:x2[1], y2[0]:y2[1]]
mask_tissue3 = mask.T[x3[0]:x3[1], y3[0]:y3[1]]

fig, ax = plt.subplots()
ax.imshow(mask_tissue1.T)
plt.show()

In [ ]:
print(tissue0.shape)
print(mask_tissue0.shape)


In [ ]:
y_tissue0_mask = tissue0[mask_tissue0].mean(axis=0)
y_tissue1_mask = tissue1[mask_tissue1].mean(axis=0)
y_tissue2_mask = tissue2[mask_tissue2].mean(axis=0)
y_tissue3_mask = tissue3[mask_tissue3].mean(axis=0)

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
ax[0].plot(data.time_stamps, y_tissue0_mask, label="tissue0")
ax[0].plot(data.time_stamps, y_tissue1_mask, label="tissue1")
ax[0].plot(data.time_stamps, y_tissue2_mask, label="tissue2")
ax[0].plot(data.time_stamps, y_tissue3_mask, label="tissue3")
ax[0].legend()
ax[0].set_title("Masked")
ax[1].plot(data.time_stamps, y_tissue0, label="tissue0")
ax[1].plot(data.time_stamps, y_tissue1, label="tissue1")
ax[1].plot(data.time_stamps, y_tissue2, label="tissue2")
ax[1].plot(data.time_stamps, y_tissue3, label="tissue3")
ax[1].legend()
ax[1].set_title("Original")
plt.show()

In [ ]:
trace_tissue0_mask = apf.Beats(y=y_tissue0_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue1_mask = apf.Beats(y=y_tissue1_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue2_mask = apf.Beats(y=y_tissue2_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue3_mask = apf.Beats(y=y_tissue3_mask, t=data.time_stamps, background_correction_method="full")

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
ax[0].plot(trace_tissue0_mask.t, trace_tissue0_mask.y, label="tissue0")
ax[0].plot(trace_tissue1_mask.t, trace_tissue1_mask.y, label="tissue1")
ax[0].plot(trace_tissue2_mask.t, trace_tissue2_mask.y, label="tissue2")
ax[0].plot(trace_tissue3_mask.t, trace_tissue3_mask.y, label="tissue3")
ax[0].legend()
ax[0].set_title("Masked")
ax[1].plot(trace_tissue0.t, trace_tissue0.y, label="tissue0")
ax[1].plot(trace_tissue1.t, trace_tissue1.y, label="tissue1")
ax[1].plot(trace_tissue2.t, trace_tissue2.y, label="tissue2")
ax[1].plot(trace_tissue3.t, trace_tissue3.y, label="tissue3")
ax[1].legend()
ax[1].set_title("Original")
plt.show()